# Sorting

### Bubble Sort

```fsharp
[| X(i) <= X(i+1) |]
[| 1;3;7;4;2 |]
// ^>^? swap positions for each position
[| 1;3;7;4;2 |] // N
//     ^>^ ? yes
// etc...
[| 1;3;4;2;|7 |] // N - 1
// after 1 iteration, largest value is at the end. 
// Next iteration doesn't need to go all the way to the end
[| 1;3;2;|4;7 |] // N - 2
[| 1;2;|3;4;7 |] // N -3
[| 1;|2;3;4;7 |] // N - (N -1)
// sorted
```

Each iteration results in `N - i` remaining positions to iterate over.
> `(n(n + 1))/2 -> O(N^2 + N) -> O(N^2)`

In [30]:
// Bubble sort (sort in place algorithm)
let bubble_sort (arr : int array) =
    let len = arr.Length - 1
    let rec walk_array (old_arr : int array) (i : int) (stop : int) =
        if i < stop then
            match arr[i] with
            | x when x <= old_arr[i + 1] ->
                walk_array old_arr (i + 1) stop
            | _ ->
                let high = old_arr[i] 
                arr[i] <- old_arr[i + 1]
                arr[i + 1] <- high
                walk_array arr (i + 1) stop
        else ()
    for stop in len .. -1 .. 0 do
        walk_array arr 0 stop

let unsorted_arr = [|6; 2; 4; 3; 7; 1; 3; 88; 12; 1; 1; 2; |]
bubble_sort unsorted_arr
unsorted_arr // now sorted!

[ 1, 1, 1, 2, 2, 3, 3, 4, 6, 7, 12, 88 ]

In [29]:
// Primeagen's implementation of bubble sort
let bubble_sort (arr : int array) =
    let len = arr.Length - 1
    for i in 0 .. len do
        // never thought to use `i` in the next nested loop
        for j in 0 .. (len - 1 - i) do // -1 since indexing up 1 with arr[j + 1]
            if (arr[j] > arr[j + 1]) then
                let high = arr[j]
                arr[j] <- arr[j + 1]
                arr[j + 1] <- high

let unsorted_arr = [|6; 2; 4; 3; 7; 1; 3; 88; 12; 1; 1; 2; |]
bubble_sort unsorted_arr
unsorted_arr // now sorted!

[ 1, 1, 1, 2, 2, 3, 3, 4, 6, 7, 12, 88 ]

### Linked List
A node based data structure.
```
(A) -> (B) -> (C) -> (D) Singly Linked List
^^^ head
(A) <-> (B) <-> (C) <-> (D) Doubly Linked List
```
Insert is `O(1)` time since it's only affecting `next` and `prev` properties of nodes.
It doesn't depend on how big the input is.

For Delete, order of operations is important because we can't break the chain.
```fsharp
B.next <- C.next
D.prev <- C.prev
C.prev <- C.next <- None
```
Delete is `O(1)`

In [ ]:
// Linked list interface
type ILinkedList =
    abstract member get_length : unit -> int
    abstract member insert_at : index : int -> element : 'T -> unit
    abstract member remove : element : 'T -> unit
    abstract member remove_at : index : int -> unit
    abstract member append : item : 'T -> unit
    abstract member prepend : item : 'T -> unit
    abstract member get : index : int -> 'T

type Node<'a> = {
    Value : 'a
    Next : Node<'a> option
    Prev: Node<'a> option
}

### Queue
Built on a Singly Linked-List

```
(A) -> (B) -> (C) -> (D)
 ^                    ^
Head                 Tail
```
> Pushing and Popping are both `O(1)`

Entering a world of DSA where constraints make things fast!
The lack of features speeds up data structures.

In [12]:
type Node<'T> = {
    value : 'T
    mutable next : Node<'T> option
}

// feed values into tail
// pull values from head
// First in, First out
type Queue<'T> (element : 'T) =
    let mutable _length = 1
    let mutable _head : Node<'T> option = Some {value = element; next = None}
    let mutable _tail : Node<'T> option = _head

    member this.head with get () = _head and set (v) = _head <- v
    member this.tail with get () = _tail and set (v) = _tail <- v
    member this.length with get () = _length

    member this.enqueue (v : 'T) =
        _length <- _length + 1
        match _tail with
        | Some tail ->
            let q = { value = v; next = None }
            _tail.Value.next <- Some q
            _tail <- Some q
        | None ->
            _tail <- Some { value = v; next = None }

    member this.deque () =
        match _head with
        | Some head ->
            _head <- head.next
            _length <- _length - 1
            Some head.value
        | None -> None

    member this.peek () = 
        match _head with
        | Some head -> Some head.value
        | None -> None

printfn "testing queue"
let test_queue = Queue 1 // constructor
test_queue.enqueue(2) 
test_queue.enqueue(3)
printfn "tail: %A" test_queue.tail // view the tail
let test_peek1 = test_queue.peek () // peek value in head
printfn $"peek: {test_peek1}"
printfn "current head: \n%A" (Option.get test_queue.head) // view the head structure
let pop1 = test_queue.deque ()  // grab value
let pop2 = test_queue.deque () // grab another value
printfn $"length after deque twice: {test_queue.length}" // check length

testing queue
tail: Some { value = 3
       next = None }
peek: Some(1)
current head: 
{ value = 1
  next = Some { value = 2
                next = Some { value = 3
                              next = None } } }
length after deque twice: 1
